In [1]:
!pip install -U bitsandbytes
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 11.5 MB/s eta 0:00:00


In [ ]:
import torch
import gc
import math
from datasets import Dataset, load_dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, TrainerCallback
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [2]:
# load dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations", split="train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

combined_dataset.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['Context', 'Response'],
    num_rows: 3512
})

In [4]:
SYSTEM_PROMPT = (
    "You are a helpful assistant and compassionate mental health counselor."
    "Respond empathetically and thoughtfully."
)

def format_instruction(example):
  return {
      "messages": [
          {"role": "system", "content": SYSTEM_PROMPT},
          {"role": "user", "content": example["Context"]},
          {"role": "assistant", "content": example["Response"]}
        ]
      }

In [5]:
final_dataset = dataset.map(format_instruction)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [6]:
final_dataset

Dataset({
    features: ['Context', 'Response', 'messages'],
    num_rows: 3512
})

In [7]:
final_dataset["messages"][0]

[{'content': 'You are a helpful assistant and compassionate mental health counselor.Respond empathetically and thoughtfully.',
  'role': 'system'},
 {'content': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
  'role': 'user'},
 {'content': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that

In [8]:
# load domain-adapted finetuned model
model_id = "Subi003/Qwen3-1.7B-Qlora8bit-MentalHealth"

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    attn_implementation="eager",
    device_map="auto",
    trust_remote_code=True
)

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.config.use_cache = False

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

adapter_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [9]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=2048, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(


In [10]:
# Convert messages to Qwen chat-formatted text
def apply_chat_template(example):
  text = tokenizer.apply_chat_template(
  example["messages"],
  tokenize=False,
  add_generation_prompt=False
  )

  return {"text": text}

In [11]:
final_dataset = final_dataset.map(
    apply_chat_template,
    remove_columns=["Context", "Response", "messages"]
)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [12]:
final_dataset

Dataset({
    features: ['text'],
    num_rows: 3512
})

In [13]:
print(final_dataset[0]["text"])

<|im_start|>system
You are a helpful assistant and compassionate mental health counselor.Respond empathetically and thoughtfully.<|im_end|>
<|im_start|>user
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?<|im_end|>
<|im_start|>assistant
<think>

</think>

If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media.  Maybe read some of the ones which state that no person is worthless, an

In [ ]:
split_ds = final_dataset.train_test_split(test_size=0.1, seed=42)

final_dataset = DatasetDict({
    "train": split_ds["train"],
    "eval": split_ds["test"]
})

In [15]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3160
    })
    eval: Dataset({
        features: ['text'],
        num_rows: 352
    })
})

## Training

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:
# LoRA setup
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [17]:
print(type(model))

<class 'transformers.models.qwen3.modeling_qwen3.Qwen3ForCausalLM'>


In [18]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
# Apply LoRA for training
lora_model = prepare_model_for_kbit_training(model)
lora_model = get_peft_model(lora_model, lora_config)

lora_model.print_trainable_parameters()

trainable params: 3,211,264 || all params: 1,723,786,240 || trainable%: 0.1863


In [20]:
class ClearCudaCacheCallback(TrainerCallback):
  def on_epoch_end(self, args, state, control, **kwargs):
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
training_args = TrainingArguments(
    output_dir="./qwen-inst-sft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    optim="adamw_torch",
    weight_decay=0.01,
    adam_epsilon=1e-10,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    max_grad_norm=1.0,
    warmup_ratio=0.05,
    lr_scheduler_type="cosine",
    do_eval=True,
    eval_steps=None,
    logging_steps=22,
    logging_first_step=True,
    save_total_limit=2,
    save_safetensors=True,
    report_to="none"
)

In [ ]:
trainer = SFTTrainer(
    model=lora_model,
    train_dataset=final_dataset["train"],
    eval_dataset=final_dataset["eval"],
    args=training_args,
    callbacks=[ClearCudaCacheCallback()]
)

In [25]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
1,3.898000
22,3.322300
44,2.414000
66,2.319400
88,2.339400
110,2.327300
132,2.308000
154,2.304200
176,2.297300
198,2.289900


TrainOutput(global_step=198, training_loss=2.4370516839653553, metrics={'train_runtime': 4476.0609, 'train_samples_per_second': 0.706, 'train_steps_per_second': 0.044, 'total_flos': 1.0797687425445888e+16, 'train_loss': 2.4370516839653553})

In [ ]:
metrics = trainer.evaluate()

eval_loss = metrics["eval_loss"]
perplexity = math.exp(eval_loss)

print(f"Eval loss: {eval_loss:.4f}")
print(f"Perplexity: {perplexity:.2f}")

Eval loss: 2.3015
Perplexity: 9.99


In [28]:
from huggingface_hub import login
login()

In [29]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_folder(
    folder_path="/content/qwen-inst-sft/checkpoint-198",
    repo_id="Subi003/Qwen3-1.7B-Qlora8bit-MentalHealth-instruct",
    repo_type="model"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...eckpoint-198/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

  ...ckpoint-198/rng_state.pth:  77%|#######7  | 11.3kB / 14.6kB            

  ...kpoint-198/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...adapter_model.safetensors:   3%|3         |  201kB / 6.45MB            

  ...eckpoint-198/optimizer.pt:   3%|3         |  406kB / 13.0MB            

  ...int-198/training_args.bin:   3%|3         |   193B / 6.22kB            

CommitInfo(commit_url='https://huggingface.co/Subi003/Qwen3-1.7B-Qlora8bit-MentalHealth-instruct/commit/843b50f58b3bbe9efb92b818027e884ef617c213', commit_message='Upload folder using huggingface_hub', commit_description='', oid='843b50f58b3bbe9efb92b818027e884ef617c213', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Subi003/Qwen3-1.7B-Qlora8bit-MentalHealth-instruct', endpoint='https://huggingface.co', repo_type='model', repo_id='Subi003/Qwen3-1.7B-Qlora8bit-MentalHealth-instruct'), pr_revision=None, pr_num=None)

## Inference

In [30]:
model_path = "/content/qwen-inst-sft/checkpoint-198"

In [ ]:
trained_model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [46]:
prompt = "I am feeling so lonely, and depressed lately."

inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = trained_model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.15,
    no_repeat_ngram_size=3,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

I am feeling so lonely, and depressed lately. I've been crying all the time since my mom died last year. She was a very loving woman who raised me on her own as an only child before she passed away from cancer. She had no one else in the family to raise me but myself.
What can I do?
Thank you for your help!
Lately, this is what's happening with us: We have a lot of people in our lives that are good, some bad ones. But we're not able to get along well or really talk about it because there are other problems going on too. This isn't necessarily true every day - sometimes things don't go smoothly between those close friends; however, these issues seem to be getting worse each day. As a result, they aren't talking much anymore.
The problem has gotten to the point where even though we know how awful life feels right now, we don't want to leave the house together any more than we need to.
In addition to having difficulty communicating effectively, there seems to be another underlying reason 

In [42]:
prompt = """what is stress?"""

inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = trained_model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

what is stress? Stress is the result of a person's perception of a situation that causes them to feel overwhelmed and unable to cope with the demands. It is a psychological state that is triggered by an event that is perceived as threatening or challenging, and it results in an individual's ability to cope with that event. Stress is a natural reaction to any event that is perceived as threatening or challenging, but the level of stress can vary greatly depending on the individual's perception and ability to cope with the event. A person who perceives a situation as threatening will have a high level of stress, while a person who perceives a situation as manageable will have a low level of stress. The stress response includes a number of physiological and psychological reactions, including increased heart rate, increased blood pressure, and increased respiration. Stress can be both a cause of illness and a way of coping with it. It is a natural reaction to any event that is perceived as

In [44]:
prompt = """We lost the tournament finals again this year after trying so hard. Feeling so bad about myself and feels like committing suicide.
What to do?"""

inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = trained_model.generate(
    **inputs,
    max_new_tokens=256,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.15,
    no_repeat_ngram_size=3,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

We lost the tournament finals again this year after trying so hard. Feeling so bad about myself and feels like committing suicide. 
What to do? I'm in college, and I haven't done anything wrong.
You're not alone. Tournaments are tough for everyone! It's normal to feel sad sometimes when things don't go your way. If you are feeling suicidal thoughts it might be helpful to talk with someone who can support you through difficult times.

It may also help if you have a mental health professional or counselor that will listen without judgment. There are many resources available online and over-the-phone as well. You can call 1-800-962-7345 (National Suicide Prevention Lifeline) or text "HELLO" to 741744 (Suicide & Crisis Lifeline). 

If you need immediate care from a medical provider, please contact the nearest emergency room or 拨打急救电话. 

Remember that even though you were upset by losing the final match, there was no reason for you to commit suicide because of what happened at the game. Ple

In [47]:
prompt = """I keep blaming myself for not being productive enough. Even when I rest, I feel guilty.
I know that rest is important, but emotionally I feel like I’m failing. Why does this guilt feeling happens, and how to overcome?
"""

inputs = tokenizer(prompt, return_tensors="pt").to(device)

outputs = trained_model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.15,
    no_repeat_ngram_size=3,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

I keep blaming myself for not being productive enough. Even when I rest, I feel guilty. 
I know that rest is important, but emotionally I feel like I’m failing. Why does this guilt feeling happens, and how to overcome?
My answer: When you do things on a schedule, it’s easy to forget the importance of taking breaks. But if your work isn’t finished in time, then it makes sense to take some time off.
The first step toward overcoming guilt about rest is knowing what kind of person you are. Are you someone who gets very stressed or anxious? If so, don’t worry too much because most people can handle short periods of stress without problems. However, long-term anxiety could lead to depression, which would make resting even harder. In addition, certain personality types may be more likely than others to have issues with guilt over their ability to rest and relax. For example, perfectionists usually suffer from an overwhelming sense of self-worth based upon whether they’ve achieved all the goal